In [ ]:
import csv
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import os

# Define file paths
students_file = 'Students.csv'
faculty_file = 'Teachers.csv'
output_file = r"D:\University\Assignments\4th Semester\AI\Project\exam_management_report.csv"

# Initialize student counts for each department
student_counts = {'CS': 0, 'EE': 0, 'AI': 0, 'BBA': 0}

# Read the student data from CSV file
with open(students_file, 'r') as student_data:
    csv_reader = csv.DictReader(student_data)
    for row in csv_reader:
        # Update student count based on department
        department = row['Dept']
        if department in student_counts:
            student_counts[department] += 1

# Display the count of students in each department
for dept, count in student_counts.items():
    print(f"{dept}: {count}")

# Load data into pandas DataFrame
students_data = pd.read_csv(students_file)
faculty_data = pd.read_csv(faculty_file)

# Define the number of rooms and their capacities
total_rooms = 30
room_sizes = [30] * 25 + [25] * 5  # 25 rooms with 30 seats, 5 rooms with 25 seats

# Define the exam schedule
exam_days = {
    'Monday': 'CS',
    'Tuesday': 'EE',
    'Wednesday': 'AI',
    'Thursday': 'BBA'
}

# One-hot encode the categorical features
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(students_data[['Batch', 'Dept']]).toarray()

# Apply K-means clustering to group students
kmeans = KMeans(n_clusters=len(exam_days), n_init=10, random_state=42)
students_data['Group'] = kmeans.fit_predict(encoded_features)

# Generate the seating plan and allocate faculty members
seating_and_faculty_plan = []

for day, dept in exam_days.items():
    students_in_dept = students_data[students_data['Dept'] == dept]
    unique_clusters = students_in_dept['Group'].unique()
    room_counter = 1

    for cluster in unique_clusters:
        clustered_students = students_in_dept[students_in_dept['Group'] == cluster]
        room_size_index = 0

        for i in range(0, len(clustered_students), room_sizes[room_size_index]):
            room_students = clustered_students.iloc[i:i + room_sizes[room_size_index]]
            assigned_faculty = faculty_data[faculty_data['Dept'] == dept].sample(1)

            for _, student in room_students.iterrows():
                seating_and_faculty_plan.append({
                    'Room': f'Room {room_counter}',
                    'Group': cluster,
                    'Batch': student['Batch'],
                    'Department': student['Dept'],
                    'Exam Day': day,
                    'Student Name': student['Name'],
                    'Student ID': student['ID'],
                    'Faculty': assigned_faculty['Name'].values[0]
                })

            room_counter += 1
            room_size_index = (room_size_index + 1) % len(room_sizes)

# Save the generated plan to a CSV file
plan_df = pd.DataFrame(seating_and_faculty_plan)
plan_df.to_csv(output_file, index=False)

# Open the generated CSV file
os.startfile(output_file)
